In [50]:
import pandas as pd
import os
from IPython.display import display, Markdown, HTML
%matplotlib inline
import tkinter as tk
from tkinter import filedialog, messagebox
from openpyxl import load_workbook
root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

''

In [51]:
messagebox.showinfo('OnSSET', 'Open the input file with calibrated GIS data')
input_file = filedialog.askopenfilename()
df = pd.read_csv(input_file)

First, reset all values from original calibration. Restart from here if calibration values below are too low/high.

In [52]:
df['ElecPopCalib'] = df['PopStartYear']
df['ElecStart'] = 0
df['Elec_Initial_Status_Grid2018'] = 0
df['Elec_Init_Status_Offgrid2018'] = 0
df['Actual_Elec_Status_2018'] = 0
df['FinalElecCode2018'] = 99

First identify grid-connected settlements

In [40]:
min_ntl_grid = 0.3
max_grid_dist = 5
min_grid_pop = 10

In [41]:
df.loc[(df['PopStartYear'] > min_grid_pop) & 
       (df['NightLights'] > min_ntl_grid) & 
       (df['CurrentMVLineDist'] < max_grid_dist), 'ElecStart'] = 1

df.loc[(df['PopStartYear'] > min_grid_pop) & 
       (df['NightLights'] > min_ntl_grid) & 
       (df['CurrentHVLineDist'] < max_grid_dist), 'ElecStart'] = 1

Next, identify off-grid settlements

In [42]:
min_off_grid_pop = 10
min_off_grid_ntl = 0.30

In [43]:
df.loc[(df['PopStartYear'] > min_off_grid_pop) & 
       (df['NightLights'] > min_off_grid_ntl) & 
       (df['CurrentMVLineDist'] > max_grid_dist) & 
       (df['CurrentHVLineDist'] > max_grid_dist) & 
       (df['HubDist'] < 3000), 'ElecStart'] = 1

In [53]:
df.loc[df['ElecStart'] == 0, 'ElecPopCalib'] = 0

In [54]:
print('Elec rate: ' + str(df['ElecPopCalib'].sum()/df['PopStartYear'].sum()*100))

Elec rate: 0.0


When the calibrated electrification rate is deemed okay, update the neccessary values

In [46]:
df.loc[(df['ElecStart'] == 1) & (df['CurrentMVLineDist'] < max_grid_dist), 'Elec_Initial_Status_Grid2018'] = 1
df.loc[(df['ElecStart'] == 1) & (df['CurrentHVLineDist'] < max_grid_dist), 'Elec_Initial_Status_Grid2018'] = 1

df.loc[(df['ElecStart'] == 1) & (df['CurrentMVLineDist'] < max_grid_dist), 'Actual_Elec_Status_2018'] = 1
df.loc[(df['ElecStart'] == 1) & (df['CurrentHVLineDist'] < max_grid_dist), 'Actual_Elec_Status_2018'] = 1

df.loc[(df['ElecStart'] == 1) & (df['CurrentMVLineDist'] < max_grid_dist), 'FinalElecCode2018'] = 1
df.loc[(df['ElecStart'] == 1) & (df['CurrentHVLineDist'] < max_grid_dist), 'FinalElecCode2018'] = 1

In [47]:
df.loc[(df['ElecStart'] == 1) & (df['CurrentMVLineDist'] > max_grid_dist) & (df['CurrentHVLineDist'] > max_grid_dist),
       'Elec_Init_Status_Offgrid2018'] = 1

df.loc[(df['ElecStart'] == 1) & (df['CurrentMVLineDist'] > max_grid_dist) & (df['CurrentHVLineDist'] > max_grid_dist), 
       'Actual_Elec_Status_2018'] = 1

df.loc[(df['ElecStart'] == 1) & (df['CurrentMVLineDist'] > max_grid_dist) & (df['CurrentHVLineDist'] > max_grid_dist), 
       'FinalElecCode2018'] = 5

In [55]:
scenario_name = 'Yemen_Calibrated_Clean_Slate'

In [56]:
messagebox.showinfo('OnSSET', 'Browse to the folder where you want to save the outputs')
df.sort_index(inplace=True)
output_dir = filedialog.askdirectory()
output_dir_results = os.path.join(output_dir, '{}.csv'.format(scenario_name))
df.to_csv(output_dir_results, index=False)